# Sytax Tree Builder
### This is an notebook to research all properties of syntax trees, including:
- Finding association rules

- Filling in the blank on missing word classes

- Building trees

In [28]:
import re
import nltk
import math
import requests
import ast
from bs4 import BeautifulSoup
from PyDictionary import PyDictionary

# Pre-X-Bar Theory Implementation
## This was made for the simpler units of LING486

In [ ]:
x = 'The big man from n loving bagels with n'
#'The cat killed the dirty dog in the house because he threw nasty bread by the fire'
#"in the smelly and wholesome and brilliant and bubbly couch"
#"the brilliant and bubbly and whimsical ball"
p = nltk.parse.api.ParserI
_d=PyDictionary()

#Array forces ordering
syntaxRulesReverse = [
    ['(adjconj)+adj', 'adj'], 
    ['(advP)*adj', 'ajP'], 
    ['(advP)?adv', 'avP'], 
    ['PnP', 'pP'],
    ['(D)?(ajP)*N(pP)*(CP)?', 'nP'], 
    ['V(nP)+(pP)*', 'vP'], 
    ['(Nconj)+N', 'N'],
    ['(nPconj)+nP', 'nP'],
    ['nPvP', 'tP'],
    ['(tPconj)+tP', 'tP']]

In [ ]:
_x = x.split()
_xd = ""
for token in _x:
    _xd += getClass(token)
_xd

In [ ]:
tree = [_xd]
diagram = []
while needsParsing(tree[-1]):
    level = ""
    dlvl = []
    for rule in syntaxRulesReverse:
        m = re.search(rule[0], tree[-1])
        if m:
            level = tree[-1][0:m.start()] + rule[1] + tree[-1][m.end():]
            tree += [level]
            dlvl += [rule[1], tree[-1][m.start():m.end()], m.start(), m.end()]
            diagram += [dlvl]
            break
    if level == "":
        break
    print('.', end='')
[print(_, end='\n|\n') for _ in tree];None

In [ ]:
_search = re.search(tree[-2], syntaxRulesReverse[-1][0])
tree[-2][0:_search.start()] + syntaxRulesReverse[-1][1] + tree[-2][_search.end():]

In [ ]:
def needsParsing(x):
    if x != 'nP' and x != 'tP' and x != 'vP' and x != 'CP' and x!= 'pP':
        return True
    return False
def getClass(x):
    wC = 'X'
    #Meaning -- regular
    word = _d.meaning(x)
    if word is not None:
        wC = list(word.keys())[0]
    
    #Meaning -- google
    word = _d.googlemeaning(x)
    if word is not None:
        wC =  word.split(':')[1].split()[0]

    if wC.lower() == 'noun' or wC.lower() == 'pronoun':
        return 'N'
    elif wC.lower() == 'verb':
        return 'V'
    elif wC.lower() == 'adjective':
        return 'adj'
    elif wC.lower() == 'adverb':
        return 'adv'
    elif wC.lower() == 'determiner':
        return 'D'
    elif wC.lower() == 'preposition':
        return 'P'
    elif wC.lower() == 'conjunction':
        return 'conj'
        
    return wC

# X-Bar and Array Tree Data Structures

In [ ]:
### Getting parts of speach using wordsmyth
word = "cover"
url = "https://www.wordsmyth.net/?level=3&ent={}".format(word)
s = requests.Session()
res = s.get(url, verify = False)
soup = BeautifulSoup(res.text, 'html5lib')
soup.findAll("table", {"class": "maintable"})[0].find_all('td', {"class": "data"})[0].a.text

In [30]:
def saveDict(d, filename):
    sD = str(d)
    f = open(filename, 'w')
    f.write(sD)
    f.close()
    
def loadDict(filename):
    f = open(filename, 'r')
    sD = f.read()
    f.close()
    d = ast.literal_eval(sD)
    return d

In [34]:
#temporary static dict
'''
localBank = {'is': 'v',
             'need' : 'v',
             'needs' : 'v',
             'do': 'v',
             'does': 'v',
             'I' : 'pn',
             'a' : 'd',
             'the' : 'd',
            #homework specific - no working suplementary dict
            'well': 'adj',
            'plays': 'v'}
'''

localBank = loadDict('localBank.dat')

In [40]:
saveDict(localBank, 'localBank.dat')

In [38]:
def getLikelyClass(word, convert = False):
    if word in localBank:
        return localBank[word]
    url = "https://www.wordsmyth.net/?level=3&ent={}".format(word)
    s = requests.Session()
    res = s.get(url, verify = False)
    soup = BeautifulSoup(res.text, 'html5lib')
    txt = soup.findAll("table", {"class": "maintable"})[0].find_all('td', {"class": "data"})[0].a.text
    switch = {
        'determiner'          :        'd',
        'noun'                :        'n',
        'pronoun'             :        'pn',
        'preposition'         :        'p',
        'transitive verb'     :        'v', #vt
        'intransitive verb'  :         'v', #vi
        'ditransitive verb'  :         'v', #vd
        'adjective'           :        'adj',
        'adverb'              :        'adv',
        'conjunction'         :        'conj',
    }
    if convert and txt in switch:
        localBank[word] = switch[txt]
        return switch[txt]
    return txt

def defineLikelyClass(word, wordClass, force=False):
    if word not in localBank or force:
        localBank[word] = wordClass
        return True
    return False

In [39]:
defineLikelyClass('\'s', 'd', True)
getLikelyClass('\'s', True)

'd'

In [48]:
#For machine gunning words into the local word bank!
x = 'The dog is in the house'.split()
y = 'd n v p d n'.split()
for i in range(len(x)):
    defineLikelyClass(x[i], y[i], True)

In [60]:
sentence = "The dog is in the house"
#"The coach will be using a new approach" #<-- HARD

#This will build all of the possible base cases for perspective
#based on research forensics for each given word
def constructPerspective(sentence):
    perspective = []
    tokens = [[_, getLikelyClass(_.lower(), True)] for _ in sentence.split(" ")]
    for token in tokens:
        perspective.append([token[1], [token[0]]])
    return perspective
x = constructPerspective(sentence)
print(x)

[['d', ['The']], ['n', ['dog']], ['v', ['is']], ['p', ['in']], ['d', ['the']], ['n', ['house']]]


In [45]:
#determines if the given rule is at the top of a given tree
#Strict disallows the tree from having different neighbors at the top
def inTreeTop(tree, rule, strict = False):
    retVal = False
    strictVal = True
    for topNode in tree:
        if topNode[0] == rule:
            retVal = True
        else:
            strictVal = False
    if strict:
        return (retVal, strictVal)
    return retVal
def pruneTrees(trees):
    pruned = []
    for tree in trees:
        finishedVal = inTreeTop(tree, "cP", strict = True)
        if finishedVal[0] and finishedVal[1]:
            pruned.append(tree)
    return pruned

In [56]:
#Debugging handling, small but out of the tree builder to keep things clean
def dprint(statement, currentLevel, evalLevel):
    if currentLevel >= evalLevel or currentLevel == -1:
        print(statement)

In [103]:
#I am Michael, I love Raquel, I eat protien, etc...
rules = {
    "vP" : [["vbar"], ["vP", "conj", "vP"]],
    "vbar" : [["vbar", "pP"], ["vbar", "advP"], ["advP", "vbar"], ["vbar", "dP"], ["v"], ["v", "dP"], ["v", "adjP"], ["v", "vP"]],
    "adjP" : [["adjbar"]],
    "adjbar" : [["adj"], ["advP", "adjbar"]],
    "advP" : [["advbar"]],
    "advbar" : [["adv"], ["advP", "advbar"]],
    "nbar": [["adjP", "nbar"], ["nbar", "pP"], ["n"], ["n", "pP"], ["pn"]],
    "nP" : [["nbar"], ["nP", "conj", "nP"]],
    "dbar" : [["d", "nP"], ["nP"]],
    "dP" : [["dP", "dbar"], ["dbar"], ["dP", "conj", "dP"]],
    "pbar" : [["p", "dP"]],
    "pP" : [["pbar"]],
    "negbar" : [["neg", "vP"]],
    "negP" : [["negbar"]],
    "t" : [["v"]],
    "tbar" : [["vP"], ["t", "vP"], ["t", "negP"]],
    "tP" : [["dP", "tbar"], ["tP", "conj", "tP"]],
    "cbar" : [["c", "tP"], ["tP"]],
    "cP" : [["cbar"], ["cP", "conj", "cP"]]
}

#This is the big boy itself
#The rule above define what the builder will try
#As of right now, the builder has a few problems
#It refuses to accept verb complements
#It does not account for null items
#As a result it doesn't yet generate movement
#
#Verboseness is for debugging purposes, it works in levels
#0 - Print nothing, smooth, user experience
#1 - Prints things like how many trees are being considered in any one permutation of rules
#2 - ...
#3 - ...
#... - ...
#-1 - PRINTS. EVERYTHING. Be careful what you wish for!!!
def BuildSyntaxTree(x, maxNode = "cP", verbose = 0):
    #Keep track of where matches for each rule occurs, when creating a new node,
    #select the simplest matches first (adjbar before tbar), and the match that occurs later in the tree
    mutating = True
    
    #This is temporary for now -- in the future we will have multiple perspectives, not just one!
    perspectives = [x]
    #Tag each perspective with an initial ID
    for i in range(len(perspectives)):
        perspectives[i] = ('{}'.format(i) , perspectives[i])
    new_perspectives = []
    exhaustive_perspectives = []
    #Measures tree formation forward and backward respectively
    #Both will (likly) be helpful -- tree formations are *messy*
    growthTree, shrinkTree = {}, {}
    while mutating:
        new_perspectives = []
        for _inp in perspectives:
            inpID, inp = _inp[0], _inp[1]
            matches = {}
            for rule, conditionSet in rules.items():    
                    for conditions in conditionSet:
                        conditionMatch, conditionPosition = 0, 0
                        dprint(rule + " : " + str(conditions), verbose, 9)
                        for i in range(len(inp)):
                            node = inp[i]
                            if node[0] == conditions[conditionMatch]:
                                dprint(rule + ", " + node[0], verbose, 10)
                                if conditionMatch == 0:
                                    conditionPosition = i
                                conditionMatch += 1
                                if conditionMatch == len(conditions):
                                    if rule in matches:
                                        matches[rule] += [(conditionPosition, conditionPosition + conditionMatch)]
                                    else:
                                        matches.update({rule:[(conditionPosition, conditionPosition + conditionMatch)]})
                                    conditionMatch = 0
                            else:
                                conditionMatch = 0
            #print("{} -> {}".format(inp, matches))
            for matchLabel, matchSets in matches.items():
                for matchSet in matchSets:
                    #print("{} :- {}".format(matchLabel, matchSet))
                    subset = inp[matchSet[0]:matchSet[1]]
                    _new_perspective = inp[:matchSet[0]] + [[matchLabel, subset]] + inp[matchSet[1]:]
                    newID = inpID + "->" + matchLabel
                    if _new_perspective in exhaustive_perspectives:
                        newID = '{}'.format(exhaustive_perspectives.index(_new_perspective))
                    else:
                        new_perspective = (newID, _new_perspective)
                        new_perspectives.append(new_perspective)
                        exhaustive_perspectives.append(_new_perspective)
                    
                    #growthTree
                    if inpID in growthTree:
                        growthTree[inpID].append(newID)
                    else:
                        growthTree[inpID] = [newID]
                    
                    #shrinkTree
                    if newID in shrinkTree:
                        shrinkTree[newID].append(inpID)
                    else:
                        shrinkTree[newID] = [inpID]
                    
                    #if new_perspective not in exhaustive_perspectives: #new_perspectives:
                    #    new_perspectives.append(new_perspective)
            for perspective in perspectives:
                if perspective[1] not in exhaustive_perspectives:
                    exhaustive_perspectives.append(perspective[1])
                if inTreeTop(perspective[1], maxNode, strict = True) == (True, True) and perspective not in new_perspectives:
                    new_perspectives.append(perspective)
        #Stops searching
        if perspectives == new_perspectives or len(new_perspectives) == 0:
            for perspective in perspectives:
                growthTree[perspective[0]] = exhaustive_perspectives.index(perspective[1])
            mutating = False
            break
        perspectives = new_perspectives.copy()
        dprint(len(perspectives), verbose, 1)
    #Returns a tuple of:
    #Final list of highest level trees
    #All generated trees
    #A dictionary denoting tree changes from beginning to end
    #A dictionary denoting tree changes from end to beginning
    return (perspectives, exhaustive_perspectives, growthTree, shrinkTree)

In [104]:
perspectives = BuildSyntaxTree(constructPerspective(sentence), verbose = 1)

4
8
15
26
36
43
48
49
44
36
27
21
16
10
6
4
2
1


In [113]:
len(perspectives[3])
perspectives[1].index(perspectives[0][0][1])
perspectives[2]['394']

KeyError: '394'

In [ ]:
final_perspectives = pruneTrees(perspectives)

In [ ]:
print("Trees: {}".format(len(final_perspectives)))
print(treeHeight(final_perspectives[0]))
final_perspectives[1]

In [ ]:
latexKeys = {
    "xbar" : '$\\bar{{{}}}$'
}

def printTree(tree):
    if type(tree) == str:
        return tree + " "
    t = tree[0]
    if type(t) == str:
        return t + " "
    label = t[0]
    if len(label) == 1:
        label = label.upper()
    ploc = label.find('P')
    if ploc > 0:
        if len(label[:ploc]) == 1:
            label = label.upper()
    barloc = label.find('bar')
    if barloc > 0:
        sublabel = label[:barloc]
        if len(sublabel) == 1:
            sublabel = sublabel.upper()
        label = latexKeys["xbar"].format(sublabel)
    if len(t[1]) == 1:
        return '[.{} {}]'.format(label, printTree(t[1]))
    return '[.{} \n\t{} \n\t{}]'.format(label, printTree([t[1][0]]), printTree([t[1][1]]))
    
print(printTree(final_perspectives[1]))

In [ ]:
#Gives the height of a tree, for those from perspective
def treeHeight(tree):
    if type(tree) == str:
        return 0
    if len(tree) == 1:
        if type(tree[0]) == str:
            return 0
    return 1+max([treeHeight(_[1]) for _ in tree])
    #return max([(0 if len(_) == 1 else max(treeHeight(__) for __ in _[1]) if (type(_) == list) else 0) for _ in tree]) + 1

#i = -10
#print(final_perspectives[0])#(perspectives[i])
#treeHeight(final_perspectives[0])#(perspectives[i])

In [ ]:
for matchLabel, matchSets in matches.items():
    for matchSet in matchSets:
        print("{} :- {}".format(matchLabel, matchSet))
        subset = inp[matchSet[0]:matchSet[1]]
        print(inp[:matchSet[0]] + [[matchLabel, subset]] + inp[matchSet[1]:])

#subset = inp[matches[matchLabel][0][0]:matches[matchLabel][0][1]]
#print(inp[:matches[matchLabel][0][0]] + [[matchLabel, subset]] + inp[matches[matchLabel][0][1]:])


In [ ]:
from ete2 import Tree
t = Tree( "((a,b),c);" )
t.render("mytree.png", w=183, units="mm")